In [ ]:
!ls -la .

In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/output'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/OnlpLab/Hebrew-Sentiment-Data.git

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertModel, BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("onlplab/alephbert-base", num_labels=3)
model.save_pretrained("./initial_pretrained")

In [ ]:
!ls -latr ./initial_pretrained

In [ ]:
import pandas as pd
dataset = {
            "name": "Heb Poems",
            "train_path": "../input/poetry/train.csv",
            "dev_path": "../input/poetry/validation.csv",
            "test_path": "../input/poetry/test.csv",
            'classes': ['0','1']
          }

def read_data():
    train = pd.read_csv(dataset['train_path'])
    dev = pd.read_csv(dataset['dev_path'])
    test = pd.read_csv(dataset['test_path'])
    return train, dev, test
train, dev, test = read_data()
train_encodings = tokenizer(train["content"].to_list(), truncation=True, padding=True)
dev_encodings = tokenizer(dev["content"].to_list(), truncation=True, padding=True)
test_encodings = tokenizer(test["content"].to_list(), truncation=True, padding=True)
train_labels=train["p_death_israel"].to_list()
dev_labels=dev["p_death_israel"].to_list()
test_labels=test["p_death_israel"].to_list()

import torch

class HebrewPoemsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HebrewSentimentDataset(train_encodings, train_labels)
dev_dataset = HebrewSentimentDataset(dev_encodings, dev_labels)
test_dataset = HebrewSentimentDataset(test_encodings, test_labels)

In [ ]:
!pip install wandb

In [ ]:
from transformers import Trainer,TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=dev_dataset             # evaluation dataset
)

trainer.train()
trainer.save_model("./alephbert_sentiment")

In [ ]:
!ls -latr ./alephbert_sentiment/

In [ ]:
import numpy as np

In [ ]:
raw_pred, _, _ = trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

In [ ]:
y_pred

In [ ]:
test_dataset.labels

In [ ]:
count_equals=0
for a,b in zip(test_dataset.labels, y_pred):
    if a==b:
        count_equals+=1
print(f"accuracy={count_equals/len(y_pred)}")